In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType , DoubleType, DateType,StructType,StructField

In [3]:
spark=SparkSession.builder.appName("streamspark").getOrCreate()

In [4]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [6]:
sample_df= spark.read.format('csv')\
.option("header", "true")\
.option('infereSchema', 'true')\
.load("/content/sample_data/kospi/KOSPI_STOCK_1.csv")

### Create the schema of the streamed files (check the column names and types from the CSV files)

In [7]:
sample_df.show(3)

+---+----------+------------+------------+------------+------------+------------+------+
|_c0|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|
+---+----------+------------+------------+------------+------------+------------+------+
| 40|2000-02-29|25163.699219|26087.699219|24239.599609|25519.099609|24115.490234|233246|
| 41|2000-03-01|25519.099609|25519.099609|25519.099609|25519.099609|24115.490234|     0|
| 42|2000-03-02|25767.900391|29144.300781|25767.900391|28575.699219|27003.972656|408391|
+---+----------+------------+------------+------------+------------+------------+------+
only showing top 3 rows



In [8]:
sample_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)



In [9]:
new_schema=StructType([StructField('ID', IntegerType(), True),
                           StructField('Date', DateType(), True),
                           StructField('Open', DoubleType(), True),
                           StructField('High', DoubleType(), True),
                           StructField('Low', DoubleType(), True),
                           StructField('Adj', DoubleType(), True),
                           StructField('Close', DoubleType(), True),
                           StructField('Volume', IntegerType(), True)])

### Create the dataframe by reading the stream using format "csv" and the schema you created.

In [10]:
new_df=spark.readStream.format('csv')\
.schema(new_schema)\
.load("/content/sample_data/kospi")


### Make sure the sataframe is streaming the files from the folder

In [11]:
new_df.isStreaming

True

In [12]:
new_df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Adj: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



### Create a stream writer into memory and specify the query name "stock:

In [13]:
stream_writer=new_df.writeStream.outputMode('append')\
            .format('memory')\
            .queryName('stock')

### Start the write stream and make sure it works (read all columns from the table)

In [14]:
query = stream_writer.start('stock')

In [15]:
spark.sql('select * from stock').show(10)

+----+----------+------------+------------+------------+------------+------------+------+
|  ID|      Date|        Open|        High|         Low|         Adj|       Close|Volume|
+----+----------+------------+------------+------------+------------+------------+------+
|null|      null|        null|        null|        null|        null|        null|  null|
| 240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797| 91019|
| 241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791|
| 242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656|
| 243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964|
| 244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|
| 245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328| 74560|
| 246|2000-12-13|27651.599609|     29286.5|27651.599609|28469.099609|26903.234375|270385|
| 247|2000

+----+----------+------------+------------+------------+------------+------------+------+
|  ID|      Date|        Open|        High|         Low|         Adj|       Close|Volume|
+----+----------+------------+------------+------------+------------+------------+------+
|null|      null|        null|        null|        null|        null|        null|  null|
| 240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797| 91019|
| 241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791|
| 242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656|
| 243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964|
| 244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|
| 245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328| 74560|
| 246|2000-12-13|27651.599609|     29286.5|27651.599609|28469.099609|26903.234375|270385|
| 247|2000

### Remove the first row from the data (hint: drop the rows where ALL values are null), then add a new column "diff", which is the difference between high and low columns

In [16]:
df2=new_df.dropna(how='all')

In [17]:
modified_data=df2.withColumn("diff",df2['high']-df2['low'])

### Create a new write stream using the new generated dataframe and call the generate table "modified_data"

In [19]:
stream_writer=modified_data.writeStream.outputMode('append')\
            .format('memory')\
            .queryName('stock')

In [20]:
query.stop()

In [21]:
query =stream_writer.start()

In [23]:
df3=spark.sql('select * from stock')

### Write the generated data into files instead of the memory. 

In [24]:
df3.write.format('parquet').save('gen2')

### Stop the query. Now, try reading the generated parquet files into a normal dataframe
- Create a schema and use it to read the data.
- Show the output.

In [25]:
query.stop()

In [30]:
finalDF= spark.read.parquet('gen2' , header = True)

In [31]:
finalDF.show(5)

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| ID|      Date|        Open|        High|         Low|         Adj|       Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797| 91019|  995.201172000001|
|241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791| 710.8007819999984|
|242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656| 533.1015620000035|
|243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964| 1066.298827999999|
|244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|1208.4003909999992|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
only showing top 5 rows



### Sort the dataframe based on the ID

In [32]:
finalDFSorted = finalDF.sort('ID')
finalDFSorted.show()

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| ID|      Date|        Open|        High|         Low|         Adj|       Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|  0|2000-01-04|22817.900391|25696.800781|22817.900391|24879.300781|23510.880859|108745|2878.9003900000025|
|  1|2000-01-05|24523.900391|26229.900391|23670.900391|24417.300781|23074.294922|175990|            2559.0|
|  2|2000-01-06|24381.699219|24666.099609|22746.800781|22817.900391|21562.865234| 71746| 1919.298827999999|
|  3|2000-01-07|     22036.0|24879.300781|     22036.0|23884.199219|22570.513672|120984|2843.3007810000017|
|  4|2000-01-10|24879.300781|25519.099609|23813.099609|24061.900391|22738.439453|151371|            1706.0|
|  5|2000-01-11|     24168.5|     25021.5|23955.199219|24239.599609|22906.365234| 95943|1066.3007810000017|
|  6|2000-01-12|     24168.5